In [5]:
import torch
import torchvision.models as models
import os
from torch import nn

In [208]:
model = models.mobilenet_v2(weights='DEFAULT')

In [13]:
def print_model_size(mdl):
    torch.save(mdl.state_dict(), "tmp.pt")
    print("%.2f MB" %(os.path.getsize("tmp.pt")/1e6))
    os.remove('tmp.pt')

In [210]:
print_model_size(model)

14.24 MB


In [211]:
model.eval()

x = model(torch.randn(1,3,224,224))

print(x.shape)

torch.Size([1, 1000])


In [212]:
model_dynamic_quantized = torch.quantization.quantize_dynamic(
    model, qconfig_spec={torch.nn.Linear}, dtype=torch.qint8
)

In [213]:
print_model_size(model_dynamic_quantized)

10.40 MB


In [215]:
model_dynamic_quantized.eval()

x = model_dynamic_quantized(torch.randn(1,3,224,224))

print(x.shape)

torch.Size([1, 1000])


In [216]:
traced_net = torch.jit.trace(model_dynamic_quantized, torch.randn(1,3,28,28))

torch.jit.save(traced_net,'DynQntMobileNet.pt')

In [163]:
print(model)

QuantizableMobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm

In [47]:
model.eval()

# Run the model on an input image
input_image = torch.randn(1, 3, 224, 224)
output = model(input_image)

# Print the output
print(output.shape)

torch.Size([1, 1000])


In [98]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.models as models

In [99]:
class MobileNet(torch.nn.Module):
    def __init__(self):
        super(MobileNet, self).__init__()
        self.model = models.quantization.mobilenet_v2(pretrained=True, quantize=True)
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()
        
        
        for param in self.model.parameters():
            param.requires_grad = False
        
        self.model.classifier[1] = nn.Sequential(
            nn.quantized.Linear(in_features=self.model.classifier[1].in_features,out_features=2))

    def forward(self, x):
        x = self.quant(x)
        x = self.model(x)
        x = self.dequant(x)
        return x




In [100]:
model = MobileNet()

/home/ss/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ss/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_QuantizedWeights.IMAGENET1K_QNNPACK_V1`. You can also use `weights=MobileNet_V2_QuantizedWeights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/ss/miniconda3/lib/python3.12/site-packages/torch/ao/quantization/utils.py:339: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


In [101]:
backend = 'x86'
model.qconfig = torch.quantization.get_default_qat_qconfig(backend)
model_qat = torch.quantization.prepare_qat(model, inplace=False)

/home/ss/miniconda3/lib/python3.12/site-packages/torch/ao/quantization/observer.py:220: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [102]:
print_model_size(model)

2.34 MB


In [6]:
class QnMobileNet(torch.nn.Module):
    def __init__(self):
        super(QnMobileNet, self).__init__()
        
        self.model = models.quantization.mobilenet_v2(pretrained=True, progress=True, quantize=True)
        
        # for param in self.model.parameters():
        #     param.requires_grad = False
        
        self.model.classifier[1] = nn.Sequential(
            nn.quantized.Linear(in_features=self.model.classifier[1].in_features,out_features=2))

        # print(self.model)

    def forward(self, x):
        
        x = self.model(x)
        
        return x

In [7]:
model = QnMobileNet()

/home/ss/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ss/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_QuantizedWeights.IMAGENET1K_QNNPACK_V1`. You can also use `weights=MobileNet_V2_QuantizedWeights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/ss/miniconda3/lib/python3.12/site-packages/torch/ao/quantization/utils.py:339: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


In [8]:
print_model_size(model)

NameError: name 'print_model_size' is not defined

In [206]:
# model.eval()

x = model(torch.randn(1,3,224,224))


In [207]:
traced_net = torch.jit.trace(model, torch.randn(1,3,28,28))

torch.jit.save(traced_net,'QuantizedMobileNet.pt')

In [9]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.models as models



class MobileNet(torch.nn.Module):
    def __init__(self):
        super(MobileNet, self).__init__()
        self.model = models.mobilenet_v2(weights='MobileNet_V2_Weights.DEFAULT')
        
        
        for param in self.model.parameters():
            param.requires_grad = False
            
        
        self.model.classifier[1] = nn.Sequential(
            nn.Linear(in_features=self.model.classifier[1].in_features,out_features=512),
            nn.LeakyReLU(negative_slope=0.02,inplace=True),
            nn.BatchNorm1d(num_features=512),
            nn.Dropout(p=0.4,inplace=True),
            nn.Linear(in_features=512,out_features=2),
            nn.Softmax(dim=1))
        
        # print(self.model)

    def forward(self, x):
        x = self.model(x)
        return x

In [10]:
def load_model():
    model = MobileNet()
    state_dict = torch.load('DogCatMobileNetv2.pt')
    model.load_state_dict(state_dict)
    model.to('cpu')
    return model

In [23]:
model = torch.jit.load('DogCatMobileNetv2.pt')

In [24]:
print_model_size(model)

11.76 MB


In [25]:
backend = 'x86'
model.qconfig = torch.quantization.get_default_qat_qconfig(backend)
quantized_model = torch.ao.quantization.convert(model.eval(), inplace=False)
quantized_model.eval()

RecursiveScriptModule(
  original_name=MobileNet
  (model): RecursiveScriptModule(
    original_name=MobileNetV2
    (features): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=Conv2dNormActivation
        (0): RecursiveScriptModule(original_name=Conv2d)
        (1): RecursiveScriptModule(original_name=BatchNorm2d)
        (2): RecursiveScriptModule(original_name=ReLU6)
      )
      (1): RecursiveScriptModule(
        original_name=InvertedResidual
        (conv): RecursiveScriptModule(
          original_name=Sequential
          (0): RecursiveScriptModule(
            original_name=Conv2dNormActivation
            (0): RecursiveScriptModule(original_name=Conv2d)
            (1): RecursiveScriptModule(original_name=BatchNorm2d)
            (2): RecursiveScriptModule(original_name=ReLU6)
          )
          (1): RecursiveScriptModule(original_name=Conv2d)
          (2): RecursiveScriptModule(original_name=BatchNorm2d)
   

In [26]:
print_model_size(quantized_model)

11.76 MB


In [27]:
quantized_model = torch.quantization.quantize_dynamic(
    model, {nn.Linear}, dtype=torch.qint8
)

In [28]:
print_model_size(quantized_model)

11.76 MB


In [29]:
print(quantized_model)

RecursiveScriptModule(
  original_name=MobileNet
  (model): RecursiveScriptModule(
    original_name=MobileNetV2
    (features): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=Conv2dNormActivation
        (0): RecursiveScriptModule(original_name=Conv2d)
        (1): RecursiveScriptModule(original_name=BatchNorm2d)
        (2): RecursiveScriptModule(original_name=ReLU6)
      )
      (1): RecursiveScriptModule(
        original_name=InvertedResidual
        (conv): RecursiveScriptModule(
          original_name=Sequential
          (0): RecursiveScriptModule(
            original_name=Conv2dNormActivation
            (0): RecursiveScriptModule(original_name=Conv2d)
            (1): RecursiveScriptModule(original_name=BatchNorm2d)
            (2): RecursiveScriptModule(original_name=ReLU6)
          )
          (1): RecursiveScriptModule(original_name=Conv2d)
          (2): RecursiveScriptModule(original_name=BatchNorm2d)
   